In [1]:
from snowflake import connector
import pathlib
from dotenv import dotenv_values
import pandas as pd
from sqlalchemy import create_engine
import os

In [2]:
# Get the parent directory of the current working directory
script_path = pathlib.Path.cwd().parent
config = dotenv_values(f"{script_path}/configuration.env")

In [3]:
# Create the connection URL with proper formatting
connection_url = (
    f"snowflake://{config.get('snowflake_user')}:"
    f"{config.get('snowflake_password')}@"
    f"{config.get('snowflake_account')}/"
    # f"{config.get('snowflake_database')}/"
    # f"{config.get('snowflake_schema')}?"
    f"warehouse={config.get('snowflake_warehouse')}&"
    f"role={config.get('snowflake_role')}"
)

engine = create_engine(connection_url)

In [6]:
# Load data with pandas
dashboard_query = """
/* =============================================================
   COMPLETE, FLATTENED VIEW FOR DASHBOARD CONSUMPTION
   ============================================================= */

SELECT
    /* ───────────── FACT COLUMNS ───────────── */
    f.review_id,
    
    /* ───────────── CUSTOMER DIM ───────────── */
    c.customer_name,
    c.nationality,
    c.number_of_flights,

    /* ───────────── REVIEW-DATE DIM (all columns) ───────────── */
    f.date_submitted_id,
    -- ds.day_of_week            AS review_day_of_week,
    -- ds.day_of_week_name       AS review_day_of_week_name,
    -- ds.cal_week_start_date    AS review_cal_week_start_date,
    -- ds.day_of_month           AS review_day_of_month,
    -- ds.cal_month              AS review_cal_month,
    -- ds.cal_mon_name           AS review_cal_mon_name,
    -- ds.cal_mon_name_short     AS review_cal_mon_name_short,
    -- ds.cal_quarter            AS review_cal_quarter,
    -- ds.cal_quarter_name       AS review_cal_quarter_name,
    -- ds.cal_year               AS review_cal_year,
    -- ds.is_weekend             AS review_is_weekend,
    -- ds.fin_year               AS review_fin_year,
    -- ds.fin_period             AS review_fin_period,
    -- ds.fin_quarter            AS review_fin_quarter,
    -- ds.fin_week               AS review_fin_week,
    -- ds.fin_period_name        AS review_fin_period_name,
    -- ds.fin_quarter_name       AS review_fin_quarter_name,
    -- ds.fin_week_name          AS review_fin_week_name,

    /* ───────────── FLIGHT-DATE DIM (all columns) ───────────── */
    f.date_flown_id,
    -- df.day_of_week            AS flight_day_of_week,
    -- df.day_of_week_name       AS flight_day_of_week_name,
    -- df.cal_week_start_date    AS flight_cal_week_start_date,
    -- df.day_of_month           AS flight_day_of_month,
    -- df.cal_month              AS flight_cal_month,
    -- df.cal_mon_name           AS flight_cal_mon_name,
    -- df.cal_mon_name_short     AS flight_cal_mon_name_short,
    -- df.cal_quarter            AS flight_cal_quarter,
    -- df.cal_quarter_name       AS flight_cal_quarter_name,
    -- df.cal_year               AS flight_cal_year,
    -- df.is_weekend             AS flight_is_weekend,
    -- df.fin_year               AS flight_fin_year,
    -- df.fin_period             AS flight_fin_period,
    -- df.fin_quarter            AS flight_fin_quarter,
    -- df.fin_week               AS flight_fin_week,
    -- df.fin_period_name        AS flight_fin_period_name,
    -- df.fin_quarter_name       AS flight_fin_quarter_name,
    -- df.fin_week_name          AS flight_fin_week_name,

    /* ───────────── ORIGIN LOCATION DIM ───────────── */
    f.origin_location_id,
    ol.city                   AS origin_city,
    ol.airport                AS origin_airport,

    /* ───────────── DESTINATION LOCATION DIM ───────────── */
    f.destination_location_id,
    dl.city                   AS destination_city,
    dl.airport                AS destination_airport,

    /* ───────────── TRANSIT LOCATION DIM ───────────── */
    f.transit_location_id,
    tl.city                   AS transit_city,
    tl.airport                AS transit_airport,


    /* ───────────── AIRCRAFT DIM ───────────── */
    f.aircraft_id,
    a.aircraft_model,
    a.aircraft_manufacturer,
    a.seat_capacity,

    /* ───────────── REVIEW CONTEXT ───────────── */
    f.verified,
    f.seat_type,
    f.type_of_traveller,

    /* ───────────── METRICS ───────────── */
    f.seat_comfort,
    f.cabin_staff_service,
    f.food_and_beverages,
    f.inflight_entertainment,
    f.ground_service,
    f.wifi_and_connectivity,
    f.value_for_money,
    f.average_rating,
    f.rating_band,
    f.recommended,

    /* ───────────── REVIEW_TEXT ───────────── */
    f.review_text,

    /* ───────────── TIMESTAMPS ───────────── */
    f.el_updated_at,
    f.t_updated_at

FROM british_airways_db.marts.fct_review_enriched  AS f

/* ----- date dimensions ----- */
LEFT JOIN british_airways_db.marts.dim_date     AS ds
       ON f.date_submitted_id = ds.date_id

LEFT JOIN british_airways_db.marts.dim_date     AS df
       ON f.date_flown_id     = df.date_id

/* ----- customer dimension ----- */
LEFT JOIN british_airways_db.marts.dim_customer AS c
       ON f.customer_id = c.customer_id

/* ----- location dimensions (three roles) ----- */
LEFT JOIN british_airways_db.marts.dim_location AS ol
       ON f.origin_location_id = ol.location_id

LEFT JOIN british_airways_db.marts.dim_location AS dl
       ON f.destination_location_id = dl.location_id

LEFT JOIN british_airways_db.marts.dim_location AS tl
       ON f.transit_location_id = tl.location_id

/* ----- aircraft dimension ----- */
LEFT JOIN british_airways_db.marts.dim_aircraft AS a
       ON f.aircraft_id = a.aircraft_id

/* optional filter */
WHERE f.date_submitted_id IS NOT NULL
ORDER BY 1 DESC
"""
df = pd.read_sql(dashboard_query, engine)

df.head()

,review_id,review_date_id,review_day_of_week,review_month_name,review_month_number,review_year,review_verified,customer_name,flight_month_name,flight_month_number,...,aircraft_manufacturer,aircraft_seat_capacity,review_seat_type,origin_city,origin_airport,destination_city,destination_airport,transit_city,transit_airport,review_text
0,1,2015-08-23,Sun,August,8,2015,False,Colin Pay,August,8,...,Airbus,134.0,Economy Class,London,LHR,Dublin,DUB,Unknown,Unknown,No problems on this short flight. Left and arr...
1,2,2015-08-25,Tue,August,8,2015,False,Desmond Jones,August,8,...,Unknown,NaN,Economy Class,London,LHR,Copenhagen,CPH,Unknown,Unknown,They sold me tickets without any luggage allow...
2,3,2015-08-25,Tue,August,8,2015,False,Kenneth Barton,August,8,...,Airbus,134.0,Economy Class,London,LHR,Toulouse,TLS,Unknown,Unknown,This was my first flight on a British Airways ...
3,4,2015-08-26,Wed,August,8,2015,False,A Coogans,August,8,...,Airbus,180.0,Economy Class,Glasgow,GLA,Prague,PRG,London,LHR,Check in agent at Glasgow was very cheerful an...
4,5,2015-08-26,Wed,August,8,2015,False,C Johnson,August,8,...,Unknown,NaN,Economy Class,Denver,DEN,London,LHR,Unknown,Unknown,"Slight delay. Games advertised on website, not..."


In [7]:
df.columns

Index(['review_id', 'review_date_id', 'review_day_of_week',
       'review_month_name', 'review_month_number', 'review_year',
       'review_verified', 'customer_name', 'flight_month_name',
       'flight_month_number', 'flight_year', 'flight_month_year',
       'customer_country', 'aircraft_model', 'aircraft_manufacturer',
       'aircraft_seat_capacity', 'review_seat_type', 'origin_city',
       'origin_airport', 'destination_city', 'destination_airport',
       'transit_city', 'transit_airport', 'review_text'],
      dtype='object')

In [8]:
df.to_csv("../data/review.csv", index=False)